<a href="https://colab.research.google.com/github/lblogan14/Python_Deep_Learning/blob/master/ch8_rl_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RL Paradigms
* **Agent**: The Entity for which is learned actions.
* **Environment**: The world in which the agent operates.
* **State**: All of the information available to the agent about its current environment.
* **Action**: A possible response, or set of responses, an agent can take. After each action, the environment will change its state and then provide feedback to the agent.
* **Reward**: The feedback that the agent gets from the environment after each action. The agent's main objective is to maximize the total return (accumulated rewards) in the long run.
* **Policy**: Determines what actions the agent will take, given the current state. In the course of the training, the agent will try to modify its policy to make better decisions. The task of finding the optimal policy is called policy improvement (or **control**) and is one of the major RL tasks.
* **Value function**: Determines what is good for the agent in the long run (unlike the immediate reward). The agent's policy will take into consideration the value - and, to a lesser extent, the reward - when deciding what action to take. The task of finding the value function is called **prediction** (also known as policy evaluation) and is the other major RL task.

##Types of RL Algorithms
* **Tabular solutions**: The number of possible states and actions is small enough to represent the value function as a table (array) and the agent is fully familiar with the environment. With tabular solutions, the true optimal value function and potimal policy can be found.
* **Approximate solutions**: The state and action spaces could be arbitrarily large. On top of that, the large number of possible states means that the agent will inevitably run into situations it has never seen before. The way to solve a problem such as this is to find an approximation of the value function, which can also generalize over the unseen data. Fortunately, deep neural networks have proven to be good candidates for that role.

###Types of RL agents
* **Value-based agents**: These store the value function and base their decisions on it. Such an agent will decide which action to take based on the value of the states, where the action leads. These agents don't use a policy.
* **Policy-based agents**: These use only the policy, and not the value function, when deciding what action to take.
* **Actor-Critic agents**: Use both the value function and the policy to make decisions.
* **Model-based agents**： These include a model of the environment. Given a state and an action, the agent can use the model as a simulation of the real environment to predict the next state and reward. In other words, the model allows the agent to plan its future actions.
* **Model-free agents**: These don't have an internal model of the environment, and learn the policy with a trial-and-error approach. Model-free agents learn to take their future actions.

RL agents that use policy can be further classified:
* **On-policy**: The agent takes actions based on the current policy.
* **Off-policy**: The agent bases its actions on a **behavior policy**, while it tries to optimize another target policy.

#RL as a Markov Decision Process
A **Markov decision process (MDP)** is a mathematical framework for modeling decisions. Assume to work with a full knowledge of the environment. An MDP provides a formal definition of the following properties:
* $\mathcal{S}$ is the finite set of all possible environment states, and $s_t$ is the state at time $t$.
* $\mathcal{A}$ is the set of all possible actions, and $a_t$ is the action at time $t$.
* $\mathcal{P}$ is the dynamics of the environment (also known as transition probabilities matrix). It defines the conditional probability of transitioning to a new state $s'$, given the existing state $s$, and an action $a$ (for all states and actions）：
$$\mathcal{P}_{ss'}^{a}=Pr(s_{t+1}=s'|s_t=s,a_t=a)$$
These probabilities represent the model of the environment. Model-based agents ahve an internal representation of $\mathcal{P}$ using to predict the results of their actions. The current state completely characterizes the total state of the environment, which makes the MDP a memoryless process, which is called the **Markov property**.
* $\mathcal{R}$ is the reward function. It describes the reward, the agent would receive when it takes action $a$ and transition from $s\rightarrow s'$:
$$\mathcal{R}_{ss'}^{a}=\mathbb{E}[r_{t+1}|s_{t+1}=s',s_t=s, a_t=a]$$
$\mathbb{E}$ pops up because the environment is stochastic and the transitions between different states are described by the transition probabilities.
* $\gamma$ is the discount factor. It is a value in the [0,1] range and determines how much the algorithm values the immediate rewards, as opposed to the future rewards. 

![](https://github.com/lblogan14/Python_Deep_Learning/blob/master/img/ch8/mdp_example.png?raw=true)
Step-by-step execution of one MDP episode (an episode might be one game):
1. The episode starts with the initial state, $s_0$, at time $t=0$ and ends at $t=T$ when the terminal state is reached.
2. Repeat unitl the terminal state is reached:
    1. The agent takes the action $a_t$ action.
    2. The environment samples a reward $r_{t+1}$ and a new state $s_{t+1}$ based on the transitional probability $\mathcal{P}_{s_t s_{t+1}}^{a_t}=Pr(s_{t+1}|s_t,a_t)$
    3. The agent receives the new state $s_{t+1}$ and reward $r_{t+1}$.


The MDP together with the agent will produce the following sequence (or trajectory):
$$s_0, a_0, r_1, s_1, a_1, r_2, s_2, a_2, r_3, ..., a_{r-1}, r_r, s_r$$

The total return (the sum of all future rewards) is discounted with the discount factor $\gamma$. The formula for the **discounted return** at time step $t$ is given by
$$G_t=r_{t+1}+ \gamma r_{t+2} + \gamma^2 r_{t+3} + ... = \sum_{j=0}^{T}\gamma^j r_{t+j+1}$$
This can be rewritten with a recursive fashion:
$$G_t = r_{t+1} + \gamma G_{t+1}$$

The value function estimates the cumulative future rewards of a given state. The rewards depend on the future actions of the agent which are determined by the agent's policy (denoted by $\pi$). *The policy maps the state $s$ to a probability of selecting each possible action $a$, starting from $s$. There are two types of value functions:
* **State-value function** $v_{\pi}(s)$ describes the expected returns starting from the $s_t$ state and then following policy $\pi$:
$$v_{\pi}(s)=\mathbb{E}_{\pi}[G_t|s=s_t$]=\mathbb{E}_{\pi}\left[\sum_{j=0}^{T}\gamma^j r_{t+j+1}|s=s_t\right]$$
Since both the environment transition function and the policy might act in a stochastic way, the expectation $\mathbb{E}[\cdot] is used.
* **Action-value function$* $q_{\pi}(s,a)$ or **q-function** describes the expected return starting from $s$, then taking action $a$, and following policy $\pi$:
$$q_{\pi}(s,a)=\mathbb{E}_{\pi}[G_t|s=s_t, a=a_t]=\mathbb{E}_{\pi}\left[\sum_{j=0}^{T}\gamma^j r_{t+j+1}|s=s_t, a=a_t\right]$$

Denote the probability that a policy $\pi$ selects an action $a$, given a current state, $s$. Then,
$$v_{\pi}(s)=\sum_a \pi(a|s)q_{\pi}(s,a)$$
The state-value function iss equivalent to the sum of the action-value functions of all outgoing (from $s$) actions $a$ multiplied by the policy probability of selecting each action. The sum of probabilities of all outbound actions from $s$ is $\sum_a\pi(a|s)=1$.

##Bellman equations
The recursive definition of $G_t$,
$$v_{\pi}(s)=\mathbb{E}_{\pi}[G_t|s=s_t]=\mathbb{E}_{\pi}[r_{t+1}+\gamma G_{t+1}|s=s_t]$$

The expectation for the immediate reward $r_{t+1}$,
$$\mathbb{E}_{|pi}[r_{t+1}|s=s_t]=\sum_a \pi(a|s) \sum_{s'}\mathcal{P}_{ss'}^{a}\mathcal{R}_{ss'}^{a}$$
where $\mathcal{P}_{ss'}^{a}$ is the transition probability and $\mathcal{R}_{ss'}^{a}$ is the expected reward the environment will sample, when transitioning from $s$ to $s'$ via $a$. The expected immediate reward is the sum over the product of the probabilities of the policy to select each action $a$, the transition probabilities, and the expected rewards for the transitions.

The expectation of the total discounted return,
$$\mathbb{E}_{\pi}[\gamma G_{t+1}|s=s_t]=\mathbb{E}_{\pi}[\gamma\sum_{j=0}^{T}\gamma^k r_{t+j+2}|s=s_t] = \sum_a\pi(a|s)\sum_{s'}\mathcal{P}_{ss'}^{a}\gamma\mathbb{E}_{\pi}\left[\sum_{j=0}^{T}\gamma^j r_{t+j+2}|s'=s_{t+1}\right]$$

The state-value equation can be rewritten as
$$v_{\pi}(s)=\sum_a\pi(a|s)\sum_{s'}\mathcal{P}_{ss'}^{a}\left[\mathcal{R}_{ss'}^{a}+\gamma\mathbb{E}_{\pi}[\sum_{j=0}^{T}\gamma^j r_{t+j+2}|s'=s_{t+1}]\right]$$
The innermost expectataion is equal to $v_{\pi}(s'=s_{t+1})$. Thus, the Bellman equation for the state-value function is defined as
$$v_{\pi}(s)=\sum_a\pi(a|s)\sum_{s'}\mathcal{P}_{ss'}^{a}[\mathcal{R}_{ss'}^{a}+\gamma v_{\pi}(s')]$$
as shown below

![](https://github.com/lblogan14/Python_Deep_Learning/blob/master/img/ch8/s-v_function.png?raw=true)

The Bellman equation for the action-value function is given by
$$q_{\pi}(s,a)=\mathbb{E}_{\pi}[\sum_{j=0}^{T}\gamma^j r_{t+j+1}|s_t=s,a_t=a]=\sum_{s'}\mathcal{P}_{ss'}^{a}[\mathcal{R}_{ss'}^{a}+\gamma\sum_{a'}\pi(a'|s')q_{\pi}(s',a')]$$
$$q_{\pi}(s,a)=\sum_{s'}\mathcal{P}_{ss'}^{a}[\mathcal{R}_{ss'}^{a}+\gamma v_{\pi}(s')$$
![](https://github.com/lblogan14/Python_Deep_Learning/blob/master/img/ch8/a-v_function.png?raw=true)

##Optimal policies and value functions
The goal of the agent is to maximize the total cumulative reward in the long run. The policy maximizing the total cumulative reward is called the optimal policy denoted with $\pi_*$.

The state-value and action-value functions with respect to the optimal policy $\pi_*$ are given by
$$v_*(s)=\max_{\pi}v_{\pi}(s)$$
$$q_*(s,a)=\max_{\pi}q_{\pi}(s,a)$$
The optimal action-value function can be expanded with the optimal policy $\pi_*$ in terms of $v_*(s)$:
$$q_*(s,a)=\mathbb{E}[r_{t+1}+\gamma v_*(s_{t+1})|s_t=s,a_t=a]$$
The state-value function under the optimal policy must equal the expected return for the best action starting from that state $s$,
$$v_*(s)=\max_a q_*(s,a)$$
Thus, the Bellman optimality equation for $v_*(s)$:
$$v_*(s)=\max_a q_*(s,a)=\max_a\mathbb{E}[r_{t+1}+\gamma v_*(s_{t+1})|s_t=s,a_t=a]=\max_a\sum_{s'}\mathcal{P}_{ss'}^{a}[\mathcal{R}_{ss'}^a+\gamma v_*(s')]$$
where $s_{t+1}=s'$.

Thus, the Bellman optimality equation for $q_*(s,a)$ is defined as
$$q_*(s,a)=\max_a\mathbb{E}[r_{t+1}+\gamma q_*(s_{t+1},a')|s_t=s,a_t=a]=\sum_{s'}\mathcal{P}_{ss'}^{a}[\mathcal{R}_{ss'}^{a}+\gamma\max_a q_*(s',a')]$$
where $s_{t+1}=s'$ and $a'=a_{t+1}$.

The Bellman optimality equations build upon that and provide the base for iterative approaches of finding the optimal policy.

#Finding Optimal Policies with Dynamic Programming

##Policy evaluation
The policy evaluation is also known as **prediction**. Assume that the state-value function is a table. The policy evaluation is implemented by the state-value Bellman equation.

1. Input the following:
    * The policy, $\pi$.
    * A small threshold value, $\theta$, which is used to access when to stop.
2. Initialize the following:
    * The variable $\Delta$ with 0. Combine it with $\theta$ to access whether to stop
    * The table $v_{\pi}$ with some value for all states.
3. Repeat unitl $\Delta<\theta$:
    * $\Delta=0$
    * For each state $s_i$ in $\mathcal{S}$, do the following:
        1. Extract the expected total return, $v_{s_i}=v_{\pi}(s_i)$, for $s_i$.
        2. Update the discounted return of $s_i$ with the Bellman equation:
        $$v_{\pi}(s_i)=\sum_a\pi(a|s_i)\sum_{s_i'}\mathcal{P}_{s_i s_i'}^{a}[\mathcal{R}_{s_i s_i'}^{a}+\gamma v_{\pi}(s_i')]$$
        Given a current state $s_i$, iterate over all possible actions $a$, for which the policy $\pi$, gives a non-zero probability. Then for each of these actions, calculate the sum of the reward $\mathcal{R}_{s_i s_i'}^{a}$ (transition from $s_i\rightarrow s_i'$ state) and the discounted returns $\gamma v_{\pi}(s_i')$ of the new state $s_i'$. Update the value of a state using the values of its neighboring states.
        $$\Delta=\max(\Delta, |v_{s_i}-v{\pi}(s_i)|)$$

##Policy improvements
Policy improvement is also known as **control**. Assume that the policy is represented as a table, where the best actions are stored for each state (tabular solution). Assume that a value function $v_{\pi}$ and a policy $\pi$ have been found. For each state $s$, do the following:
1. Assume all possible actions starting from $s$ are taken. Using the action-value Bellman equation, for each action, compute the expected returns if that action is taken and the policy $\pi$ is still followed.
2. Compare the expected returns of the action, selected by the policy, to the expected returns for the rest of the actions. Update the policy to take the new action if some of the newly-computed expected returns are larger than the existing policy selection.
3. Repeat the preceding steps until the policy selects the best action in all cases, that is, until it is no longer necessary to update it.

The updated policy $\pi'$ is given by
$$\pi'(s)=\arg\max_a q_{\pi}(s,a)=\arg\max_a\mathbb{E}[r_{t+1}+\gamma v(s')|s,a]=\arg\max_a\sum_{s'}\mathcal{P}_{ss'}^{a}[\mathcal{R}_{ss'}^{a}+\gamma v_{\pi}(s')]$$

##Policy and value iterations
**policy iteration** refers to alternating steps of policy evaluation and policy improvement until the process converges:
$$\pi_0\xrightarrow{\text{Eval}}v_{\pi_0}\xrightarrow{\text{Improve}}\pi_1\xrightarrow{\text{Eval}}v_{\pi_1}\xrightarrow{\text{Improve}}pi_2\xrightarrow{\text{Eval}}\cdots\xrightarrow{\text{Improve}}\pi_*\xrightarrow{\text{Eval}}v_{\pi_*}$$
One drawback of policy iteration is that it performs evaluation in each iteration.

#Monte Carlo Methods

##Policy evaluation
Describe the value function estimation $v_{\pi}$ given a policy $\pi$ in the following steps:
1. Input the policy $\pi$
2. Initialize the following:
    * The table $v_{\pi}$ with some value for all states
    * An empty list of `returns(s)` for each state $s$
3. For a number of episodes, do the following:
    1. Generate a new episode, following policy $\pi$: $s_0,a_0,r_1,s_1,a_1,r_2,s_2,a_2,r_3,...,a_{T-1},r_T,s_T$.
    2. Initialize the cumulative discounted return, $G=0$.
    3. Iterate over each $t$ step of the episode, starting from $T-1$ and going to $0$:
        1. Update $G$ with the reward at the $t+1$ step: $G=G+\gamma r_{t+1}$.
        2. If the $s_t$ state does not appear in any of the preceding episode steps $s_0, s_1, s_2,...,s_{t-1}$:
            1. Append the current value of $G$ to the `return(s_t)` list associated with $s_t$.
            2. Update the value function with the average of `return(s_t)`:
            $$v_{\pi}(s_t)=\text{average}(\text{returns}(s_t))$$

This MC variant is called **first-visit** because if a state $s$ appears multiple times in one episode, $G$ is only added to `return(s)` the first time that the state occurred in the episode trajectory and ignore other occurrences.

Another MC variant is called **every-visit**. The discounted reward is added every time the state occurs in the episode. The same pseudo-code with first-visit can be used here except that the check of whether the state has already occurred is removed. 

##Exploring starts policy improvement
MC policy improvement also follows the same general pattern as Dynamic Programming (DP), alternating evaluation and improvement steps until convergence. If the model of the environment is unknown, it is better to estimate the action-value function $q_{\pi}(s,a)$ (state-action pairs) instead of the state-value function.

Generate multiple episodess and then average the returns of each state-action pairs. If the policy is deterministic, every time the agent moves to a particular state $s$, it will choose the same action $a$. So there is a chance that some state-action pairs may never get visited and $q_{\pi}(s,a)$ for them will not be estimated. This is where the **exploring states (ES)** comes in: Each eposide starts with a state-action pair (and not just with state), and each state-action pair has non-zero probability of starting an episode.

Pseudo-code for first-visit MC ES: 
1. Input the policy $\pi$
2. Initialize the following:
    * The $q_{\pi}(s,a)$ table with some value for all state-action pairs
    * An empty `returns(s,a)` list for each state-action pairs
3. For a number of episodes, do the following:
    1. Generate a new episode, following the policy $\pi$: $s_0, a_0, r_1, s_1, a_1,r_2, s_2, a_2, r_3, ... a_{T-1}, r_T, s_T$.
    2. Initialize the cumulative discounted return $G=0$.
    3. Iterate over each t step of the episode, starting from $T-1$ and going to $0$:
        1. Update $G$ with the reward at the $t+1$ step: $G=G+\gamma r_{t+1}$.
        2. If the pair $(s_t, a_t)$ does not appear in any of the preceding episode steps $s_0,a_0,s_1,a_1,...,s_{t-1},a_t$:
            1. Append the current value of $G$ to the `returns(s_t,a_t)` list associated with $(s_t,a_t)$
            2. Update the value function with the average of `returns(s_t,a_t)`: $q_{\pi}(s_t,a_t)=\text{average}(\text{returns}(s_t,a_t))$
            3. $\pi(s_t)=\arg\max_a q_{\pi}(s_t,a)$.

##Epsilon-greedy policy improvement
MC control with a non-deterministic epsilon-greedy policy

All non-optimal actions starting from the $s$ state can be selected with minimal probability $\epsilon/|\mathcal{A}(s)|$, where $|\mathcal{A}(s)|$ is the number of actions for the $s$ state. The optimal action (selected by the greedy policy) has a probability of $1-\epsilon -\epsilon/|\mathcal{A}(s)|$ being selected.

Pseudo-code for first-visit MC $\epsilon$-greedy policy:
1. Input the policy $\pi$
2. Initialize the following:
    * The $q_{\pi}(s,a)$ table with some value for all state-action pairs
    * An empty `returns(s,a)` list for each state-action pairs
3. For a number of episodes, do the following:
    1. Generate a new episode, following the policy $\pi$: $s_0, a_0, r_1, s_1, a_1,r_2, s_2, a_2, r_3, ... a_{T-1}, r_T, s_T$.
    2. Initialize the cumulative discounted return $G=0$.
    3. Iterate over each t step of the episode, starting from $T-1$ and going to $0$:
        1. Update $G$ with the reward at the $t+1$ step: $G=G+\gamma r_{t+1}$.
        2. If the pair $(s_t, a_t)$ does not appear in any of the preceding episode steps $s_0,a_0,s_1,a_1,...,s_{t-1},a_t$:
            1. Append the current value of $G$ to the `returns(s_t,a_t)` list associated with $(s_t,a_t)$
            2. $a_*=\arg\max_a q_{\pi}(s_t,a)$
            3. For all actions $a_i$, starting from state $s$:
            $$\pi(a_i,s_t)=
            \begin{cases}
                1-\epsilon+\epsilon/|\mathcal{A}(s_t)| & \text{if }a_i=a_* \\
                \epsilon/|\mathcal{A}(s_t)| & a_i\neq a_*
            \end{cases}$$

$\epsilon$-greedy is non-deterministic (every action is selected with a probability).

#Temporal Difference Methods
**Temporal difference (TD)** is a class of model-free RL methods. They can learn from the agent's experience, such as MC, or estimate state values based on the values of other states, such as DP.

##Policy evaluation
TD methods can update the action-value function after each step of the episode. A TD algorithm uses the following to perform a state-value update:
$$v(s_t)=v(s_t)+\alpha[r_{t+1}+\gamma v(s_{t+1})-v(s_t)]$$
where $\alpha$ is the step size (learning rate) in the range of [0, 1]. Following a policy $\pi$, when the agent transits from $s_t$ to $s_{t+1}$, the agent has received a reward $r_{t+1}$.

$r_{t+1}+\gamma v(s_{t+1})-v(s_t)$ is the difference between the label and the predicted value of the algorithm (known as the TD error). This method is called **one-step TD** or **TD(0)**. Other variants include n-step TD and TD($\lambda$).

Pseudocode of TD(0):
1. Input the policy, $\pi$.
2. Initialize the $v_{\pi}$ table with some value for all states
3. Repeat for a number of episodes:
    1. Start new episode with an initial state of $s_{t=0}$
    2. Repeat until the terminal state is reached:
        1. Select action $a_t$, using the policy $\pi$ for the current state $s$.
        2. Take the action $a_t$, transition to new state $s_{t+1}$, and observe reward $r_{t+1}$.
        3. Update the value function $v(s_t)=v(s_t)+\alpha[r_{t+1}+\gamma v(s_{t+1})-v(s_t)]$
        4. $s_t=s_{t+1}$

##Control with SARSA
The name is SARSA comes from the five-element sequence of the episode's trajectory: $s_t,a_t,r_{t+1},s_{t+1},a_{t+1}$.

SARSA is an on-policy TD control method to estimate the action-value function in order to find the optimal policy. $q_{\pi}(s,a)$ is update online after each step of an episode in the iterations of multiple episodes. The update formula is
$$q(s_t,a_t)=q(s_t,a_t)+\alpha[r_{t+1}+\gamma q(s_{t+1},a_{t+1})-q(s_t,a_t)]$$
where $q_{\pi}(s_{terminal},a)=0$ for each action of the terminal state. Start with $s_t$ state, take the $a_t$ action (following the policy), transits to the next state $s_{t+1}$, and then take the next action $a_{t+1}$ (following the policy)

Pseudocode of SARSA:
1. Input the policy, $\pi$.
2. Initialize the $q_{\pi}(s,a)$ table with some value for all states
3. Repeat the following for a number of episodes:
    1. Start new episode with an initial state-action pair, $s_{t=0}, a_{a=0}$
    2. Repeat the following until the terminal state is reached:
        1. Take the action $a$, and trnasit it to a new state $s_{t+1}$, and observe reward $r_{t+1}$
        2. Select next action $a_{t+1}$ following the policy $\pi$
        3. Update the action-value function $q(s_t,a_t)=q(s_t,a_t)+\alpha[r_{t+1}+\gamma q(s_{t+1},a_{t+1})-q(s_t,a_t)]$
        4. $s_t=s_{t+1}$, $a_t=a_{t+1}$

##Control with Q-learning
Q-learning is an off-policy TD control method. The action-value function for Q-learning is defined as
$$q(s_t, a_t)=q(s_t,a_t)+\alpha[r_{t+1}+\gamma\max_a q(s_{t+1},a)-q(s_t,a_t)]$$
Since Q-learning is an off-policy method, it has two distinct policies:
* **Behavior policy**: The agent uses this to actually navigate through the environment. 
* **Target policy**: The agent uses this to compute the TD error in the action-value update function.

Pseudocode for Q-learning:
1. Input the policy, $\pi$.
2. Initialize the $q_{\pi}(s,a)$ table with some value for all state-action pairs.
3. Repeat the following for a number of episodes:
    1. Start new episode with the initial state-action pairs $s_{t=0}, a_{t=0}$.
    2. Repeat the following until the terminal state is reached:
        1. Take the action $a_t$, transit to a new state $s_{t+1}$, and observe the reward $r_{t+1}$.
        2. Select next action $a_{t+1}$ following the behavior policy (for example, $\epsilon$-greedy).
        3. Update the action-value function,
        $$q(s_t, a_t)=q(s_t,a_t)+\alpha[r_{t+1}+\gamma\max_a q(s_{t+1},a)-q(s_t,a_t)]$$
        using the greedy target policy.
        4. $s_t=s_{t+1}, a_t=a_{t+1}$

##Double Q-learning
Imagine that the majority of the actions $a$, starting from the state $s$, have true action-values $q_*(s,a)=0$. That is, the real return for each action starting from the state $s$ is 0. The estimations $q(s,a)$ are uncertain. The issue comes up when the estimation of each state-action pair using the greedy target policy is computed. The action-value of the pair with maximum expected return is slightly positive, so the estimated action-values for all pairs will be slightly higher than the real action-values, which are zeros. This issue is called **maximization bias**.

The idea of double Q-learning is to decompose the selection and evaluation in two separate action-value estimations: 
$q_1$ and $q_2$. Both will try to estimate the optimal action-value function $q_*$, but the state-action pairs are splitted in two sets - the first set to train $q_1$ and the second set to train $q_2$. Now $q_1$ is used to select the best action and $q_2$ to estimate its value. Then the update rule for $q_1$ becomes:
$$q_1(s_t,a_t)=q_1(s_t,a_t)+\alpha[r_{t+1}+\gamma q_2(s_{t+1}\arg\max_a q_1(s_{t+1},a)-q_1(s_t,a_t)]$$
$q_1$ and $q_2$ still suffer from maximization bias, but by using different training sets, at least they will overestimate different actions $a$ when starting from the same state $s$. Even if $q_1$ selects an overestimated action, the action-value $q_2$ will not be overestimated, thus minimizing the maximization bias. The formula works for $q_2$ as well.

Pseudocode for double Q-learning:
1. Input the policy, $\pi$.
2. Initialize the $q_1(s,a)$ and $q_2(s,a)$ tables with some value for all state-action pairs
3. Repeat the following for a number of episodes:
    1. Start a new episode with the initial state $s_{t=0}$.
    2. Repeat until the terminal state is reached:
        1. Select the action $a_t$ following the behavior policy based on both $q_1$ and $q_2$ (for example, $\epsilon$-greedy)
        2. Take the action $a_t$, transit to a new state $s_{t+1}$, and observe reward $r_{t+1}$.
        3. Update one of the two action-value estimations with a probability of 0.5:
        $$q_1(s_t,a_t)=q_1(s_t,a_t)+\alpha[r_{t+1}+\gamma q_2(s_{t+1}\arg\max_a q_1(s_{t+1},a)-q_1(s_t,a_t)]$$
        $$q_2(s_t,a_t)=q_2(s_t,a_t)+\alpha[r_{t+1}+\gamma q_1(s_{t+1}\arg\max_a q_2(s_{t+1},a)-q_2(s_t,a_t)]$$
        4. $s_t=s_{t+1}$

#Value Function Approximations
As actual functions $v_{\pi}$ and $q_{\pi}$ with as many parameters as the number of table cells grow, it becomes impossible to store all of them. Not only that, but with a large number of states, the agent is bound to enter situation it has never seen before. The goal now is to find another set of functions $\hat{v}$ and $\hat{q}$, with the following properties:
* Approximate $v_{\pi}$ and $q_{\pi}$ with significantly fewer parameters, compared to the tabular version
* Generalizes well enough, so they can successfully approximate previously-unseen situations

$$\hat{v}(s,\mathbf{w})\approx v_{\pi}(s)$$
$$\hat{q}(s,a,\mathbf{w})\approx q_{\pi}(s,a)$$
where $\mathbf{w}$ is the function parameters. Here, $\mathbf{w}$ means the weights of the neural networks.

* The network input is the current state or stat-action pair (depending on whether it estimates $v$ or $q$)
* The network output is the value function approximation $\hat{v}$ or $\hat{q}$.
* The target data (labels) is the real value function $v_{\pi}$ or $q_{\pi}$.

The loss function for the state- and action-value functions are defined as
$$J_v(\mathbf{w})=\frac{1}{2}\mathbb{E}_{\pi}[v_{\pi}(s)-\hat{v}(s,\mathbf{w})]^2$$
$$J_q(\mathbf{w})=\frac{1}{2}\mathbb{E}_{\pi}[q_{\pi}(s,a)-\hat{q}(s,a,\mathbf{w})]^2$$

The network parameters are updated by **stocahstic gradient descent (SGD)**:
$$\nabla_{\mathbf{w}}J_v(\mathbf{w})=-\mathbb{E}[v_{\pi}(s)-\hat{v}(s,\mathbf{w})]\nabla_{\mathbf{w}}\hat{v}(s,\mathbf{w})$$
$$\nabla_{\mathbf{w}}J_q(\mathbf{w})=-\mathbb{E}[q_{\pi}(s,a)-\hat{q}(s,a,\mathbf{w})]\nabla_{\mathbf{w}}\hat{q}(s,a,\mathbf{w})$$

The weight-update rules are,
$$\Delta\mathbf{w}=-\alpha\nabla_{\mathbf{w}}J_{v,w}(\mathbf{w})$$
$$\mathbf{w}=\mathbf{w}+(-\alpha\nabla_{\mathbf{w}}J_{v,w}(\mathbf{w})$$

Recall the state-value function update rule in the temporal difference methods:
$$v(s_t)=v(s_t)+\alpha[\underbrace{r_{t+1}+\gamma v(s_{t+1})}_\text{target}-\underbrace{v(s_t)}_\text{approx.}]$$
Similarly, the weight update here becomes:
$$\mathbf{w}=\mathbf{w}-\alpha(\underbrace{r_{t+1}+\gamma \overbrace{\hat{v}(s_{t+1},\mathbf{w})}^\text{net output t+1} }_\text{target} -\underbrace{\hat{v}(s_t,\mathbf{w})}_\text{net output t}) \nabla_{\mathbf{w}}\hat{v}(s_t,\mathbf{w})$$

The neural network is trained online using the TD algorithm (SARSA, Q-learning). The training set is the stream of (action, reward, new state)

Pseudocode:
1. Input the following:
    * Policy $\pi$
    * Value function approximator $\hat{v}$ (neural net)
2. Repeat the following steps for a number of episodes:
    1. Start a new episode with the initial state $s_{t=0}$
    2. Repeat until the terminal state is reached:
        1. Select the $a_t$ action using the policy $\pi$ for the current state $s$.
        2. Take the action $a_t$, transit to new state $s_{t+1}$, and observe reward $r_{t+1}$
        3. Update the network weights:
        $$\mathbf{w}=\mathbf{w}-\alpha(r_{t+1}+\gamma \hat{v}(s_{t+1},\mathbf{w}) -\hat{v}(s_t,\mathbf{w})) \nabla_{\mathbf{w}}\hat{v}(s_t,\mathbf{w})$$
        4. $s_t=s_{t+1}$

##Value approximation for SARSA and Q-learning
SARSA update rule:
$$q(s_t,a_t)=q(s_t,a_t)+ \alpha[r_{t+1}+\gamma q(s_{t+1},a_{t+1})-q(s_t,a_t)]$$
$$\mathbf{w}=\mathbf{w}-\alpha(\underbrace{r_{t+1}+\gamma \overbrace{\hat{q}(s_{t+1}, a_{t+1},\mathbf{w})}^\text{net output t+1} }_\text{target} -\underbrace{\hat{q}(s_t,a_t,\mathbf{w})}_\text{net output t}) \nabla_{\mathbf{w}}\hat{q}(s_t,a_t,\mathbf{w})$$

Q-learning update rule:
$$q(s_t,a_t)=q(s_t,a_t)+ \alpha[r_{t+1}+\gamma \max_a q(s_{t+1},a)-q(s_t,a_t)]$$
$$\mathbf{w}=\mathbf{w}-\alpha(\underbrace{r_{t+1}+\gamma\max_a \overbrace{\hat{q}(s_{t+1}, a,\mathbf{w})}^\text{net output t+1} }_\text{target} -\underbrace{\hat{q}(s_t,a_t,\mathbf{w})}_\text{net output t}) \nabla_{\mathbf{w}}\hat{q}(s_t,a_t,\mathbf{w})$$

#Experience Replay
The experience replay helps the network "remember" previous training data and experiences. As the environment interaction goes, a sliding window of the latests $n$ interactions is stored: (state $s_{t-1}$, action $a_{t-1}$, reward $r_t$, state $s_t$) for $t = t_{now - n} ... t_{now}$. Instead of training the network with the latest data, a mini-batch is created by extracting samples from various points of the sliding window. In this way, the network will receive diversified training data and will perform much better.